## Week 6: DNA Methylation
### Radhika Jangi 10/16/2020

### Bisulfite Mapping
SRR3083926_1.chr6.fastq from STEM-seq E4.0ICM rep1<br/>
SRR3083926_2.chr6.fastq from STEM-seq E4.0ICM rep1<br/>
SRR3083929_1.chr6.fastq from STEM-seq E5.5Epi rep1<br/>
SRR3083929_2.chr6.fastq from STEM-seq E5.5Epi rep1

bismark_genome_preparation --bowtie2 chr6<br/>

bismark --genome chr6/ -1 SRR3083926_1.chr6.fastq,SRR3083929_1.chr6.fastq -2 SRR3083926_2.chr6.fastq,SRR3083929_2.chr6.fastq<br/>

samtools sort SRR3083926_1.chr6_bismark_bt2_pe.bam -o SRR26.sorted.bam<br/>
samtools sort SRR3083929_1.chr6_bismark_bt2_pe.bam -o SRR29.sorted.bam<br/>

samtools index SRR26.sorted.bam<br/>
samtools index SRR29.sorted.bam<br/>

bismark_methylation_extractor --bedgraph --comprehensive SRR3083926_1.chr6_bismark_bt2_pe.bam<br/>
bismark_methylation_extractor --bedgraph --comprehensive SRR3083929_1.chr6_bismark_bt2_pe.bam

### Python Scripting

In [69]:
# Get bedgraph files
f = open('SRR3083926_1.chr6_bismark_bt2_pe.bedGraph', 'r')
SRR26 = f.readlines()
SRR26 = SRR26[1:] # Skip header
f.close()
fs = open('SRR3083929_1.chr6_bismark_bt2_pe.bedGraph', 'r')
SRR29 = fs.readlines()
SRR29 = SRR29[1:] # Skip header
fs.close()

In [70]:
# Get gene and coordinate files
f = open('mm10_refseq_genes_chr6_50M_60M.bed', 'r')
refseq = f.readlines()
f.close()

In [71]:
# Keeps unique genes and their coordinates
unique_genes = {}
for line in refseq:
    gene_coord = list(map(int,line.split()[4:6])) # Grabs gene coordinates from columns 5 and 6
    gene_name = line.split()[12] # Grabs gene name from column 13
    unique_genes.setdefault(gene_name,gene_coord)

In [72]:
def methylation_count(srr, search_range):
    '''Calculates average methylation score in a specified region
    INPUT: 
        - srr: bedgraph data for specific embryonic cell type
        - search_range: coordinates of specific gene within genome
    OUTPUT:
        - meth_score/count: average methylation score for a gene
        - 0: if no methylation found on gene
    '''
    meth_score = 0 # Total methylation score
    count = 0 # Counts number of non-zero methylation sites
    for line in srr:
        meth_range = list(map(int,line.split()[1:3])) # Gets location of nucleotide
        score = float(line.split()[3]) # Gets methylation score at that site
        if meth_range[1]>=search_range[0] and meth_range[1]<=search_range[1]: # Checks if nuc falls within gene range
            meth_score+= score
            count+=1
    if count != 0:
        return meth_score/count # Returns avg methylation for current gene
    else:
        return 0 # No methylation on this gene

In [73]:
# Maps genes to fold change from e4 to e5.5
f = open("foldchange.txt", "a")
fs = open("e4_avg_methylation.txt", "a")
fl = open("e5_5_avg_methylation.txt", "a")

# Loops through unique genes + coordinates and gets methylation
# For both e4 and e5 on each gene
for gene, coords in unique_genes.items():
    e4 = methylation_count(SRR26,coords)
    fs.write(gene+ ":\t"+str(e4))
    fs.write('\n')
    
    e5_5 = methylation_count(SRR29,coords)
    fl.write(gene+ ":\t"+str(e5_5))
    fl.write('\n')
    
    if e4 !=0:
    #if e5_5 !=0:
        f.write(gene+ ":\t"+str((e5_5-e4)/e4)) # Calculate fold change
        f.write('\n')
#         else:
#             f.write(gene+ ":\t"+'-1.0')
#             f.write('\n')
f.close()
fs.close()
fl.close()